In [1]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
import glob
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
import os
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [2]:
document_index = None
persist_directory = "/Users/juan/Documents/Econ_unidos"
pdf_folder="/Users/juan/Documents/Juan MacBook Pro/CUNEF/Quinto/TFM/Selectividad/Economia Unidos"

In [36]:
def init_document_index():
    global document_index 
    if document_index is None:
        print('\nInitializing Document Index')
        files = glob.glob(pdf_folder+'/*.pdf', recursive=False)
        text = ''
        for file in files:
            PyPDF2.PdfReader(file)
            for page_num in range(len(reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text() # type: ignore
        docs = re.split(r"\d\.\s", text)
        embedding = OpenAIEmbeddings()
        print('\nLoading to Chroma')
        vectordb = Chroma.from_texts(texts=docs, embedding=embedding, persist_directory=persist_directory)
        print('\nPersisting')
        vectordb.persist()
        print('\nPersisted')
        document_index = VectorStoreIndexWrapper(vectorstore=vectordb)
    return document_index

In [39]:
def get_document_index():
    global document_index
    if document_index is None:
        print()
        embedding = OpenAIEmbeddings() # type: ignore
        vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
        document_index = VectorStoreIndexWrapper(vectorstore=vectordb)
    return document_index

In [40]:
def get_document_vectordb():
    embedding = OpenAIEmbeddings() # type: ignore
    vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
    return vectordb

In [41]:
gpt4 = ChatOpenAI(model_name="gpt-4", temperature=0) # type: ignore

In [37]:
init_document_index()


Initializing Document Index


Using embedded DuckDB with persistence: data will be stored in: /Users/juan/Documents/Econ_unidos



Loading to Chroma

Persisting

Persisted


VectorStoreIndexWrapper(vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7faae944bbe0>)

In [42]:
index=get_document_index()

In [84]:
template = """Use the following pieces of context to answer the question at the end. 
In the context you will find a statement part which is the exam question, and a solution part which is the text between "SOLUCIÓN:" and the next question statement.
You should only show the solution part if requested by the prompt.
If you are asked to provide questions, print the question literally without changing the wording.

Alternatively you may be asked to give feedback on an answer to a given question.
In this case find the question being asked and compare the user respones to the solution part (text after "SOLUCION:")and correct whatever is wrong or incomplete.
Give a score out of ten to the user solution.


If you don't know the answer, just say that you don't know. Don't try to make up an answer.
Respond in Spanish.


{context}

QUESTION: {question}
=========
FINAL ANSWER IN SPANISH:"""
PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])


In [85]:
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(gpt4, chain_type="stuff",
                                 retriever=get_document_vectordb().as_retriever(),
                                 chain_type_kwargs=chain_type_kwargs, verbose = True)

Using embedded DuckDB with persistence: data will be stored in: /Users/juan/Documents/Econ_unidos


In [78]:
query = "Dame dos preguntas"
print(qa.run(query))



> Entering new RetrievalQA chain...

> Finished chain.
Indique las dos ventajas competitivas que una empresa puede conseguir mediante la inversión en I+D+i (0,5 puntos) y exponga dos ejemplos (0,5 puntos).

Teniendo en cuenta la información anterior, se pide:
a) Calcule el resultado del ejercicio 2021 de esta empresa (0,25 puntos).
b) Elabore el Balance de Situación de la empresa a 31 de diciembre de 2021, indicando las principales masas patrimoniales (1 punto).
c) Calcule los valores de las siguientes ratios de la empresa e interprete su resultado (0,75 puntos):
− Ratio de liquidez: activo corriente / pasivo corriente. Valor recomendado de referencia: entre 1,5 y


In [79]:
query = "Dame dos preguntas y sus respuestas"
print(qa.run(query))



> Entering new RetrievalQA chain...

> Finished chain.
Primera pregunta: Señale a qué tipo de organización de la empresa, formal o informal, responde la siguiente situación, argumentando su respuesta: «La persona que se incorporó a comienzos de año al departamento de expansión tiene una especial habilidad para trasmitir entusiasmo y motivación, creando un excelente ambiente de trabajo, colaboración y camaradería entre la mayor parte de la plantilla».
Respuesta: La situación expuesta responde a un tipo de organización informal, ya que ha surgido de una manera espontánea, al margen de los cauces de decisión de la empresa, no estando asociada a unidades organizativas de la misma y descansando, en cambio, sobre las relaciones que voluntariamente se gestan entre las personas que trabajan en la empresa y en los roles que estas adoptan, entre los que destaca el liderazgo de la persona del ejemplo.

Segunda pregunta: ¿A qué tipo de organización respondería la siguiente decisión de la empresa

In [87]:
query = """ 
Estoy respondiendo a la pregunta:Señale a qué tipo de organización de la empresa, formal o informal, responde la siguiente situación, argumentando su respuesta: «La persona que se incorporó a comienzos de año al departamento de expansión tiene una especial habilidad para trasmitir entusiasmo y motivación, creando un excelente ambiente de trabajo, colaboración y camaradería entre la mayor parte de la plantilla»
Y mi respuesta es:
Es una organizacion formal porque surgio espontaneamente. ademas las relaciones se gestan voluntariamente
"""
print(qa.run(query))



> Entering new RetrievalQA chain...

> Finished chain.
La respuesta es incorrecta. La situación descrita corresponde a una organización informal porque efectivamente surgió espontáneamente y se basa en las relaciones voluntarias entre las personas que trabajan en la empresa. Sin embargo, has mencionado que es una organización formal cuando en realidad es informal. Te daría un puntaje de 4/10 por reconocer el surgimiento espontáneo y las relaciones voluntarias, pero necesitas corregir la clasificación de la organización.


In [88]:
query = """ 
Hazme una pregunta sobre el punto muerto pero cambia los datos y dame la respuesta ajustada a los nuevos datos 
"""
print(qa.run(query))



> Entering new RetrievalQA chain...

> Finished chain.
Pregunta: Considerando una empresa que vende camisetas a 10 euros por unidad, con costes fijos mensuales de 5.000 euros y costes variables unitarios de 4 euros, ¿cuál es el punto muerto o umbral de ventas?

Solución: Primero calculemos el punto muerto utilizando la fórmula:

Umbral de rentabilidad = costes fijos / (precio de venta unitario - costes variables unitarios)

Umbral de rentabilidad = 5.000 / (10 - 4) = 5.000 / 6 = 833,33 unidades.

Por lo tanto, el punto muerto o umbral de ventas es de 833,33 unidades. Esto significa que la empresa necesita vender al menos 834 unidades para cubrir todos sus costes y comenzar a obtener beneficios.
